### Hotel in Jeju 
<strong>&nbsp;&nbsp;&nbsp;jeju hotel guest consumption eda</strong>
- 1급 호텔 (4,5 성급)
- 2급 호텔 (2,3 성급)
- 기타 숙박업 (1성급, 기타 표시되지 않은 숙박업종 중 기준 충족되는 업소)

<strong>&nbsp;&nbsp;&nbsp;Datas</strong>
1. 제주도 내 숙박 업종 분포 (2020.05 ~ 2020.08 기준)<br>
- 이용 횟수, 숙박 업종 내 총 매출에서 1급 호텔이 가장 낮다. <br>
2. 제주 데이터 허브 내국인 관광객 지역, 업종, 성별, 연령대별 카드 이용 데이터 (2017.01 ~ 2018.12.31)

<br>
<strong>&nbsp;&nbsp;&nbsp;Hypotheses</strong><br><br>
&nbsp;1. 1급 호텔과 다른 숙박 업종의 bill 단가를 비교했을 때 가격 차이가 크지 않을 것이다.<br>
&nbsp;2. 1급 호텔이 관광객 주 소비 지역과 가까이 위치하지 않을 것이다.<br>
&nbsp;3. 1급 호텔이 다른 숙박 업종에 비해 군집을 이루지 않았을 것이다.<br>

In [127]:
import pandas as pd
import numpy as np
from pyproj import Proj, transform

In [73]:
detailed_data_url = '../datas/detailed_consume_type_data.csv'
detailed_data = pd.read_csv(detailed_data_url, sep=',')
detailed_data.tail(5)

,YM,Sido,Sigungu,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y,LargeCategory,MediumCategory
273178,202005,제주특별자치도,제주시,중소,침구수예점,x시,17000,0,1,0,949616.6171,1502504.547,소매업,그외 기타 종합 소매업
273179,202005,제주특별자치도,제주시,중소1,서양음식,x시,12000,0,1,0,949665.1566,1502804.667,음식점업,서양식 음식점업
273180,202005,제주특별자치도,제주시,영세,편의점,x시,4800,0,1,0,950266.5976,1502457.688,소매업,체인화 편의점
273181,202005,제주특별자치도,제주시,중소,제과점,x시,25000,0,2,0,950524.6587,1500759.566,소매업,빵 및 과자류 소매업
273182,202005,제주특별자치도,제주시,영세,서양음식,x시,6000,0,1,0,950674.5989,1500760.286,음식점업,서양식 음식점업


0. 제주 내 영향력있는 업종 (업종별 소비자 수, 소비횟수, 소비금액)

In [126]:
consume_category = detailed_data.pivot_table(index='MediumCategory', values=['TotalSpent','NumofSpent', 'NumofDisSpent'])
consume_category.sort_values('TotalSpent', axis=0, ascending=False).head(10)

,NumofDisSpent,NumofSpent,TotalSpent
MediumCategory,,,
면세점,0.000000,2110.133333,1.684341e+08
항공 및 여객 운송업,0.000000,858.181818,5.955224e+07
휴양콘도 운영업,1.593156,37.570342,3.795432e+06
호텔업,0.710332,17.177122,3.785727e+06
골프장 운영업,0.000000,23.672120,2.848299e+06
기타 대형 종합 소매업,4.592398,43.997574,2.476073e+06
차량용 주유소 운영업,7.489326,50.043258,2.353151e+06
슈퍼마켓,17.312933,76.602388,1.841137e+06
기타음식료품위주종합소매업,7.245781,31.039567,1.364778e+06


In [78]:
hotel_spent = detailed_data[detailed_data['MediumCategory'] == '호텔업'] 

hotel_pivot = hotel_spent.pivot_table(index='Type', values=['DisSpent', 'NumofSpent', 'NumofDisSpent'], aggfunc=np.sum)
hotel_pivot.fillna(0)

,DisSpent,NumofDisSpent,NumofSpent
Type,,,
1급호텔,402900,9,240
2급호텔,1346300,30,579
특급호텔,26238454,346,8491


2. 제주 데이터 허브 (내국인, 관광객 업종별 소비내역)

In [2]:
consumption_data_url = '../datas/jeju_api_res_datas.csv'
consumption_data = pd.read_csv(consumption_data_url, sep=',',
                              names=['sigungu', 'marketType', 'userType', 'gender', 'userCount', 'useCount', 'useCost'])

consumption_data.rename(columns= consumption_data.iloc[0])
consumption_data.reset_index(drop=True)

,sigungu,marketType,userType,gender,userCount,useCount,useCost
0,제주시,유흥,제주도민,여,23,29,4240100
1,제주시,유흥,제주도민,남,114,190,27414000
2,제주시,유흥,제주도민,여,52,71,6848000
3,제주시,유흥,제주도민,남,259,458,91468500
4,제주시,유흥,제주도민,여,77,99,10021600
...,...,...,...,...,...,...,...
16695,서귀포시,숙박,제주도민,남,84,102,3846958
16696,서귀포시,식음료,내국인관광객,남,95,163,2656250
16697,제주시,소매,제주도민,남,806,1125,74823494
16698,제주시,문화/레져,내국인관광객,여,877,971,16823000


In [115]:
local_consumption = consumption_data[consumption_data['userType'] == '제주도민']
guest_consumption = consumption_data[consumption_data['userType'] == '내국인관광객']

local_pivot = (local_consumption.pivot_table(index='marketType', values=['userCount', 'useCount', 'useCost'], aggfunc=np.sum)
               .add_prefix('local_'))
guest_pivot = (guest_consumption.pivot_table(index='marketType', values=['userCount', 'useCount', 'useCost'], aggfunc=np.sum)
               .add_prefix('guest_'))

# 제주도민,관광객 수, 업종별 소비횟수, 소비금액
concat_df = pd.concat([local_pivot, guest_pivot], axis=1)

concat_df['total_useCost'] = concat_df['local_useCost'] + concat_df['guest_useCost']
concat_df['total_userCount'] = concat_df['local_userCount'] + concat_df['guest_useCount']
concat_df['total_useCount'] = concat_df['local_useCount'] + concat_df['guest_useCount']
concat_df

,local_useCost,local_useCount,local_userCount,guest_useCost,guest_useCount,guest_userCount,total_useCost,total_userCount,total_useCount
marketType,,,,,,,,,
교통,235596728034,4238506,2652361,199464890198,2979060,2522923,435061618232,5631421,7217566
기타,274323891778,4170118,2098008,95426976177,2010343,1238926,369750867955,4108351,6180461
문화/레져,68918393547,1888397,1214628,68955443253,1498791,1145961,137873836800,2713419,3387188
소매,273535648466,3718984,2393921,137180163858,3159035,2530423,410715812324,5552956,6878019
쇼핑,585591553849,28799681,8560011,361605687366,14583636,7206258,947197241215,23143647,43383317
숙박,31680115747,319313,224588,100204671616,1020999,781350,131884787363,1245587,1340312
식음료,529335154776,18827193,8518080,393586081344,12411585,8304869,922921236120,20929665,31238778
유흥,109368275375,951247,656706,30604269530,261881,196747,139972544905,918587,1213128


3. 제주도 내 숙박 업종 (2020.05 ~ 2020.08 기준) POINT_X, POINT_Y 좌표 위도, 경도 변환 수행

In [159]:
# 좌표 변환
def change_lat_lng(path_x, path_y):
    
    itrf_x, itrf_y = path_x, path_y
    
    proj_UTMK = Proj(init='epsg:5178') #기존 좌표계: ITRF2000
    proj_WGS84 = Proj(init='epsg:4326') #변환할 좌표계
    
    x, y = transform(proj_UTMK, proj_WGS84, itrf_x, itrf_y)
    latlng = (x, y)
    
    return latlng

In [182]:
# ITRF2000 좌표계 좌표
def get_transform_xy(raw_xy):
    transform_xy = []
    latlng_df = raw_xy
    
    for i in range(len(latlng_df)):
        raw_x = latlng_df['POINT_X'].iloc[i]
        raw_y = latlng_df['POINT_Y'].iloc[i]

        res = change_lat_lng(raw_x, raw_y)
        transform_xy.append(res)

    return transform_xy


raw_xy = hotel_spent[['POINT_X', 'POINT_Y']]
res_latlng = get_transform_xy(raw_xy)

get_x, get_y = [], []

for i in range(len(res_latlng)):
    x = str(res_latlng[i][0])
    y = str(res_latlng[i][1])
    
    get_x.append(x)
    get_y.append(y)

In [200]:
hotel_spent['Proj_X'] = get_x
hotel_spent['Proj_Y'] = get_y

hotel_spent.reset_index(drop=True, inplace=True)
hotel_spent.tail()

,YM,Sido,Sigungu,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y,LargeCategory,MediumCategory,Proj_X,Proj_Y
537,202005,제주특별자치도,서귀포시,일반,특급호텔,x시,476120,0,2,0,898476.1888,1473521.419,숙박업,호텔업,126.40802879714563,33.251012309853614
538,202005,제주특별자치도,제주시,영세,2급호텔,x시,50000,0,1,0,906345.5569,1500097.637,숙박업,호텔업,126.48973220767313,33.49139924234485
539,202005,제주특별자치도,제주시,일반,특급호텔,x시,300000,0,2,0,908678.9024,1503357.462,숙박업,호텔업,126.51451390226558,33.52099911243486
540,202005,제주특별자치도,제주시,일반,특급호텔,x시,12600,0,2,0,909535.2455,1501962.168,숙박업,호텔업,126.52387519464753,33.50848885815475
541,202005,제주특별자치도,서귀포시,일반,특급호텔,x시,39000,0,1,0,914371.1918,1473147.432,숙박업,호텔업,126.57867641959933,33.24901831049828
